In [1]:
import pandas as pd
import numpy as np
import os
import re

from utils import (
    write_list_datasets, PATH_UCR_LOCAL, ILL_FORMATED, ILL_FORMATED_DIR,
    get_fname, process_labels, get_data_labels_UCR, print_heads,
    UNKNOWN_K, TOO_MANY_LABELS, TOO_MANY_SAMPLES, INVALID
)

In [2]:
DATA_SOURCE = "UCR"
RES_DIR = f'../res/{DATA_SOURCE}/'
PATH_UCR = PATH_UCR_LOCAL

## Save list and headers of all original datasets

In [11]:
# Find all TRAIN files
# This gives the folder. Each folder d has the following structure
# - {d}/{d}_TRAIN.tsv
# - {d}/{d}_TEST.tsv
# - {d}/README.md
subdirs = [
    f.name for f in os.scandir(PATH_UCR)
    if f.is_dir() and f.name != ILL_FORMATED_DIR[:-1]
]
subdirs

['PickupGestureWiimoteZ',
 'DistalPhalanxOutlineCorrect',
 'PigArtPressure',
 'BirdChicken',
 'PLAID',
 'MixedShapesRegularTrain',
 'SemgHandMovementCh2',
 'InsectEPGSmallTrain',
 'InsectEPGRegularTrain',
 'EOGVerticalSignal',
 'Coffee',
 'DistalPhalanxTW',
 'HandOutlines',
 'Yoga',
 'Car',
 'OliveOil',
 'RefrigerationDevices',
 'GesturePebbleZ2',
 'SonyAIBORobotSurface2',
 'BME',
 'Chinatown',
 'FiftyWords',
 'CricketY',
 'ACSF1',
 'CBF',
 'DodgerLoopDay',
 'InlineSkate',
 'FreezerSmallTrain',
 'Adiac',
 'PigCVP',
 'MiddlePhalanxTW',
 'Earthquakes',
 'Haptics',
 'GesturePebbleZ1',
 'DodgerLoopWeekend',
 'Lightning7',
 'FaceAll',
 'InsectWingbeatSound',
 'Computers',
 'MixedShapesSmallTrain',
 'CricketX',
 'TwoPatterns',
 'ECG5000',
 'SemgHandSubjectCh2',
 'ShakeGestureWiimoteZ',
 'AllGestureWiimoteX',
 'UWaveGestureLibraryX',
 'Symbols',
 'UWaveGestureLibraryAll',
 'Wafer',
 'ECGFiveDays',
 'Phoneme',
 'Mallat',
 'Fish',
 'Fungi',
 'DodgerLoopGame',
 'Plane',
 'CricketZ',
 'ArrowHead'

In [4]:
all_datasets = []
for d in subdirs:
    all_datasets += [
        f"{d}/{fname}"
        for fname in os.listdir(PATH_UCR+d)
        if "TRAIN" in fname
    ]
print(len(all_datasets))
all_datasets


128


['PickupGestureWiimoteZ/PickupGestureWiimoteZ_TRAIN.tsv',
 'DistalPhalanxOutlineCorrect/DistalPhalanxOutlineCorrect_TRAIN.tsv',
 'PigArtPressure/PigArtPressure_TRAIN.tsv',
 'BirdChicken/BirdChicken_TRAIN.tsv',
 'PLAID/PLAID_TRAIN.tsv',
 'MixedShapesRegularTrain/MixedShapesRegularTrain_TRAIN.tsv',
 'SemgHandMovementCh2/SemgHandMovementCh2_TRAIN.tsv',
 'InsectEPGSmallTrain/InsectEPGSmallTrain_TRAIN.tsv',
 'InsectEPGRegularTrain/InsectEPGRegularTrain_TRAIN.tsv',
 'EOGVerticalSignal/EOGVerticalSignal_TRAIN.tsv',
 'Coffee/Coffee_TRAIN.tsv',
 'DistalPhalanxTW/DistalPhalanxTW_TRAIN.tsv',
 'HandOutlines/HandOutlines_TRAIN.tsv',
 'Yoga/Yoga_TRAIN.tsv',
 'Car/Car_TRAIN.tsv',
 'OliveOil/OliveOil_TRAIN.tsv',
 'RefrigerationDevices/RefrigerationDevices_TRAIN.tsv',
 'GesturePebbleZ2/GesturePebbleZ2_TRAIN.tsv',
 'SonyAIBORobotSurface2/SonyAIBORobotSurface2_TRAIN.tsv',
 'BME/BME_TRAIN.tsv',
 'Chinatown/Chinatown_TRAIN.tsv',
 'FiftyWords/FiftyWords_TRAIN.tsv',
 'CricketY/CricketY_TRAIN.tsv',
 'ACSF1/AC

In [5]:
list_fname = f'{RES_DIR}all_datasets-{DATA_SOURCE}.txt'
# write_list_datasets(list_fname, subdirs)

In [6]:
# print_heads(
#     fnames=subdirs, path=PATH_UCR, n_labels_max=15, n_samples_max=3500,
#     UCR = True,
# )

## Save list of datasets that are suitable for experiments

In [7]:
not_for_exp = UNKNOWN_K+TOO_MANY_LABELS+TOO_MANY_SAMPLES+INVALID

fname_exp_theory = [
    d for d in subdirs if d not in not_for_exp
]

print(len(fname_exp_theory))
fname_exp_theory

111


['PickupGestureWiimoteZ',
 'DistalPhalanxOutlineCorrect',
 'BirdChicken',
 'PLAID',
 'MixedShapesRegularTrain',
 'SemgHandMovementCh2',
 'InsectEPGSmallTrain',
 'InsectEPGRegularTrain',
 'EOGVerticalSignal',
 'Coffee',
 'DistalPhalanxTW',
 'HandOutlines',
 'Yoga',
 'Car',
 'OliveOil',
 'RefrigerationDevices',
 'GesturePebbleZ2',
 'SonyAIBORobotSurface2',
 'BME',
 'Chinatown',
 'CricketY',
 'ACSF1',
 'CBF',
 'DodgerLoopDay',
 'InlineSkate',
 'FreezerSmallTrain',
 'MiddlePhalanxTW',
 'Earthquakes',
 'Haptics',
 'GesturePebbleZ1',
 'DodgerLoopWeekend',
 'Lightning7',
 'FaceAll',
 'InsectWingbeatSound',
 'Computers',
 'MixedShapesSmallTrain',
 'CricketX',
 'TwoPatterns',
 'ECG5000',
 'SemgHandSubjectCh2',
 'ShakeGestureWiimoteZ',
 'AllGestureWiimoteX',
 'UWaveGestureLibraryX',
 'Symbols',
 'UWaveGestureLibraryAll',
 'Wafer',
 'ECGFiveDays',
 'Mallat',
 'Fish',
 'Fungi',
 'DodgerLoopGame',
 'Plane',
 'CricketZ',
 'ArrowHead',
 'ScreenType',
 'GunPointOldVersusYoung',
 'MedicalImages',
 'Mid

In [8]:
exp_theory_list_fname = f'{RES_DIR}datasets_experiments_theory-{DATA_SOURCE}.txt'
# write_list_datasets(exp_theory_list_fname, fname_exp_theory)

In [12]:
excluded = [d for d in subdirs if d in not_for_exp]
excluded_list_fname = f'{RES_DIR}datasets_excluded-{DATA_SOURCE}.txt'
write_list_datasets(excluded_list_fname, excluded)